In [1]:
from Recipe import *
import re
from konlpy.tag import Okt
from transformers import BertTokenizer, BertModel
import torch
from scipy.spatial.distance import cosine

c:\Users\Seunghwan\Desktop\final_project\Recipe.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  recipe_df = pd.read_csv("C:/Users/Seunghwan/Desktop/데이터/kr_recipe.csv")
c:\Users\Seunghwan\Desktop\final_project\Recipe.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  recipe_df[col].fillna(0, inplace=True)
c:\Users\Seunghwan\Desktop\final_project\Recipe.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment usin

In [5]:
def extraction_of_nouns(sentence):
    # 패턴
    pattern = r'\[\w+\]' # ex) [재료], [양념]

    # 패턴으로 분리
    parts = re.split(pattern, sentence)

    # 각 파트에서 양쪽 여백 제거, 비어있지 않은 리스트만 추출
    parts = [list(map(lambda x: x.strip(), part.split("|"))) for part in parts if part]

    # 2차원 리스트 평탄화
    flattend_parts = [item for part in parts for item in part]

    # Okt 형태소 분석기 로드
    okt = Okt()

    # 명사 추출
    result = []
    for part in flattend_parts:
        nouns = okt.nouns(part)
        # print(f"문장: '{part}' -> 명사: {nouns}")
        result.extend(nouns)
    
    return result

In [10]:
recipe_df['추출된명사'] = recipe_df['재료리스트'].apply(extraction_of_nouns)
recipe_df['추출된명사']

0         [어묵, 개, 김밥, 용김, 장, 당면, 움큼, 양파, 개, 당근, 개, 깻잎, 장...
1         [두부, 모, 당근, 개, 고추, 개, 브로콜리, 개, 새우, 마리, 녹말가루, 계...
2         [밥, 공기, 당근, 개, 치자, 단무지, 개, 김치, 쪽, 무순, 약간, 날치, ...
3                          [현미, 컵, 찹쌀, 컵, 호두, 물, 컵, 소금, 약간]
4         [북어, 포, 마리, 찹쌀, 가루, 간장, 설탕, 물, 파, 마늘, 참기름, 깨소금...
                                ...                        
184986    [올리브오일, 스파게티, 명란젓, 덩이, 생크림, 마늘, 톨, 양파, 개, 소금, ...
184987    [올리브오일, 스파게티, 관찰, 레, 홀, 토마토, 캔, 화이트와인, 파르, 미지,...
184988    [오징어, 젓, 무, 말랭이, 청양고추, 개, 홍, 고추, 개, 오이, 고추, 개,...
184989      [달걀, 개, 우유, 물, 맛살, 개, 당근, 개, 대파, 소금, 참치, 액, 맛술]
184990    [계란, 개, 물, 동량, 참치, 액, 쪽파, 조금, 알, 새우, 컵, 참기름, 고...
Name: 재료종합, Length: 183924, dtype: object

In [18]:
recipe_df

,요리명,조회수,추천수,스크랩수,요리타입,음식분위기,재료타입,음식타입,재료리스트,몇인분,요리난이도,요리시간,추출된명사
0,어묵김말이,10072,6,66,튀김,간식,가공식품류,디저트,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,"[어묵, 개, 김밥, 용김, 장, 당면, 움큼, 양파, 개, 당근, 개, 깻잎, 장..."
1,두부새우전,5817,3,27,부침,일상,해물류,밑반찬,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,"[두부, 모, 당근, 개, 고추, 개, 브로콜리, 개, 새우, 마리, 녹말가루, 계..."
2,알밥,6975,8,36,굽기,일상,해물류,밥/죽/떡,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,"[밥, 공기, 당근, 개, 치자, 단무지, 개, 김치, 쪽, 무순, 약간, 날치, ..."
3,현미호두죽,3339,0,11,끓이기,일상,쌀,밥/죽/떡,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,"[현미, 컵, 찹쌀, 컵, 호두, 물, 컵, 소금, 약간]"
4,북어갈비,7173,3,97,굽기,술안주,건어물류,메인반찬,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,"[북어, 포, 마리, 찹쌀, 가루, 간장, 설탕, 물, 파, 마늘, 참기름, 깨소금..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184986,명란젓크림스파게티,59,0,0,기타,손님접대,해물류,양식,[재료] 올리브오일| 스파게티면 80g| 명란젓 1덩이| 생크림 100g| 마늘 1...,1인분,아무나,NaN,"[올리브오일, 스파게티, 명란젓, 덩이, 생크림, 마늘, 톨, 양파, 개, 소금, ..."
184987,아마트리치아나스파게티,46,0,0,기타,손님접대,돼지고기,양식,[재료] 올리브오일| 스파게티면 280~300g| 관찰레 120g| 홀토마토 1캔|...,4인분,아무나,30분이내,"[올리브오일, 스파게티, 관찰, 레, 홀, 토마토, 캔, 화이트와인, 파르, 미지,..."
184988,오징어젓무침,110,0,8,무침,일상,가공식품류,김치/젓갈/장류,[재료] 오징어젓200g| 무말랭이15g| 청양고추2개| 홍고추1개| 오이고추1개|...,2인분,아무나,NaN,"[오징어, 젓, 무, 말랭이, 청양고추, 개, 홍, 고추, 개, 오이, 고추, 개,..."
184989,계란찜,170,0,6,찜,일상,달걀/유제품,밑반찬,[재료] 달걀 6개| 우유 200ml| 물 100ml| 맛살 3개| 당근 1/3개|...,3인분,아무나,30분이내,"[달걀, 개, 우유, 물, 맛살, 개, 당근, 개, 대파, 소금, 참치, 액, 맛술]"
